# Transformer麻雀AI 開発ワークフロー（ハイブリッド・安定版）

**【目的】**
Colabのディスク容量とGoogle DriveのAPI制限の両方を考慮し、ローカルディスクとDriveを併用するハイブリッド方式で、大容量データ処理を安定して実行する。

## STEP 1: 環境設定

**最初に一度だけ実行してください。**
ライブラリをインストールし、Google Driveをマウントします。
`force_remount=True` オプションで、接続が不安定になった際に確実に再接続します。

In [1]:
!pip install mahjong
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Google Drive上のプロジェクトルートディレクトリを指定
GDRIVE_PROJECT_ROOT = "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"
%cd {GDRIVE_PROJECT_ROOT}

Mounted at /content/drive
/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI


## STEP 2: データ準備（Colabローカルに展開）

**最初に一度だけ実行してください。**
Driveにある `data.zip` を、Colabの**高速なローカルディスク上**に展開します。これによりDriveのAPIエラーを回避します。

In [2]:
import os
import zipfile

ZIP_FILE_PATH_ON_DRIVE = os.path.join(GDRIVE_PROJECT_ROOT, 'data.zip')
# 【変更点】展開先をColabのローカルディスクに変更
EXTRACT_DIR_LOCAL = '/content/data_unzipped'

if not os.path.exists(EXTRACT_DIR_LOCAL) or not os.listdir(EXTRACT_DIR_LOCAL):
    print(f"Unzipping data files to Colab's local storage...")
    os.makedirs(EXTRACT_DIR_LOCAL, exist_ok=True)
    with zipfile.ZipFile(ZIP_FILE_PATH_ON_DRIVE, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_DIR_LOCAL)
    print("Unzip complete!")
else:
    print("Unzipped data directory already exists on local storage. Skipping unzip.")

print("\n--- Sample of extracted files on local storage: ---")
!ls -l {EXTRACT_DIR_LOCAL} | head -n 6

Unzipped data directory already exists on local storage. Skipping unzip.

--- Sample of extracted files on local storage: ---
total 3292
drwxr-xr-x 2 root root 3371008 Oct  1 07:52 data


## STEP 3: データセット生成（ローカル → Driveへ保存）

**このセルを繰り返し実行してください。**
ローカルの牌譜ファイルを解析し、中間データ（チャンク）を**Google Drive上**に直接保存します。中断しても、これまでの成果はDriveに保存されているので安全です。

In [4]:
import os

# 【重要】スクリプト実行前に必ずプロジェクトルートに移動する
%cd {GDRIVE_PROJECT_ROOT}

# generate_data.pyをハイブリッド用に修正します
with open('src/transformer/generate_data.py', 'r') as f:
    code = f.read()

# 【変更点】
# 入力(DATA_DIR)はColabローカル、出力(PROCESSED_DIR)はGoogle Driveを指すように書き換える
code = code.replace("DATA_DIR = '/content/data'", f"DATA_DIR = '/content/data_unzipped'")
code = code.replace("PROCESSED_DIR = '/content/processed_data'", f"PROCESSED_DIR = '{os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')}'")

with open('src/transformer/generate_data_hybrid.py', 'w') as f:
    f.write(code)

print("Starting data generation (Local Read -> Drive Write)...")
!python -m src.transformer.generate_data_hybrid

/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI
Starting data generation (Local Read -> Drive Write)...
2025-10-01 07:55:53.189316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759305353.206992    2709 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759305353.211818    2709 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759305353.226294    2709 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759305353.226349    2709 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the sam

## STEP 4: チャンク結合 ＆ 最終データセット作成（Drive上で完結）

**STEP 3で「All files have been processed」と表示されたら、このセルを一度だけ実行してください。**
Drive上に保存された中間ファイルを結合し、最終的なデータセットをDrive上に作成します。

In [3]:
import glob, pickle, numpy as np, os, tensorflow as tf
from src.transformer.vectorizer import MAX_CONTEXT_LENGTH, MAX_CHOICES

# 全てのパスをGoogle Drive上のパスに設定
GDRIVE_PROCESSED_DIR = os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data')
CHUNK_DIR_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'chunks')
FINAL_DATASET_PATH_GDRIVE = os.path.join(GDRIVE_PROCESSED_DIR, 'training_dataset_transformer.pkl')

print("--- Starting Final Merging Process (on Google Drive) ---")
%cd {GDRIVE_PROJECT_ROOT}
chunk_files = sorted(glob.glob(os.path.join(CHUNK_DIR_GDRIVE, "*.pkl")))

if not chunk_files:
    print("Error: No chunk files found on Drive to merge. Please run STEP 3 first.")
else:
    print(f"Found {len(chunk_files)} chunks on Drive. Calculating total size...")
    total_samples = sum(len(pickle.load(open(f, 'rb'))[2]) for f in chunk_files)
    print(f"Total samples to merge: {total_samples}")

    # メモリを節約するため、巨大な配列は直接作成せず、チャンクごとに追加していく
    contexts_final, choices_final, labels_final = [], [], []

    progbar = tf.keras.utils.Progbar(len(chunk_files), unit_name="chunk")
    for i, chunk_file in enumerate(chunk_files):
        with open(chunk_file, 'rb') as f:
            contexts, choices, labels = pickle.load(f)
        contexts_final.append(contexts)
        choices_final.append(choices)
        labels_final.append(labels)
        progbar.update(i + 1)

    print("\nConcatenating all chunks...")
    contexts_final = np.concatenate(contexts_final, axis=0)
    choices_final = np.concatenate(choices_final, axis=0)
    labels_final = np.concatenate(labels_final, axis=0)

    print("\nGenerating final masks...")
    non_zero_counts = np.count_nonzero(np.sum(choices_final, axis=2), axis=1)
    masks = np.zeros_like(choices_final[:, :, 0], dtype='float32')
    for i, count in enumerate(non_zero_counts): masks[i, :count] = 1.0
    final_dataset = (contexts_final, choices_final, labels_final, masks)

    print(f"Saving final dataset to Google Drive: '{FINAL_DATASET_PATH_GDRIVE}'...")
    with open(FINAL_DATASET_PATH_GDRIVE, 'wb') as f:
        pickle.dump(final_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)

    print("\n--- Merging Complete! ---")

    print("Cleaning up temporary chunk files from Google Drive...")
    !rm -rf "{CHUNK_DIR_GDRIVE}"
    print("Done.")

--- Starting Final Merging Process (on Google Drive) ---
/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI
Found 453 chunks on Drive. Calculating total size...


OSError: [Errno 107] Transport endpoint is not connected

## STEP 5: AIモデルの学習

**STEP 4が完了したら、このセルを実行してください。**
Drive上の最終データセットを使ってTransformerモデルの学習を開始します。学習が終わると、完成したモデルがDriveに保存されます。

In [ ]:
import os

# 【重要】スクリプト実行前に必ずプロジェクトルートに移動する
%cd {GDRIVE_PROJECT_ROOT}

# train_transformer.pyをDrive上で動作するように修正します
with open('src/transformer/train_transformer.py', 'r') as f:
    code = f.read()

# パス設定をGoogle Drive上のパスに書き換える
code = code.replace("PROCESSED_DATA_PATH = '/content/processed_data/training_dataset_transformer.pkl'", f"PROCESSED_DATA_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'processed_data', 'training_dataset_transformer.pkl')}'")
code = code.replace("MODEL_PATH = '/content/models/senas_jan_ai_transformer_v1.keras'", f"MODEL_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'models', 'senas_jan_ai_transformer_v1.keras')}'")
code = code.replace("os.makedirs('/content/models', exist_ok=True)", f"os.makedirs('{os.path.join(GDRIVE_PROJECT_ROOT, 'models')}', exist_ok=True)")

with open('src/transformer/train_transformer_hybrid.py', 'w') as f:
    f.write(code)

print("--- Starting Training ---")
!python -m src.transformer.train_transformer_hybrid

## STEP 6: 学習済みAIによる予測

**学習が完了したら、このセルを実行してください。**

In [ ]:
import os

# 【重要】スクリプト実行前に必ずプロジェクトルートに移動する
%cd {GDRIVE_PROJECT_ROOT}

# predict_transformer.pyをDrive上で動作するように修正します
with open('src/transformer/predict_transformer.py', 'r') as f:
    code = f.read()

# パス設定をGoogle Drive上のパスに書き換える
code = code.replace("MODEL_PATH = '/content/models/senas_jan_ai_transformer_v1.keras'", f"MODEL_PATH = '{os.path.join(GDRIVE_PROJECT_ROOT, 'models', 'senas_jan_ai_transformer_v1.keras')}'")

with open('src/transformer/predict_transformer_hybrid.py', 'w') as f:
    f.write(code)

!python -m src.transformer.predict_transformer_hybrid